In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from scipy.interpolate import griddata
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap

# Function to multiply ticks by 1000
def adjust_x_axis(x, pos):
    return "{:.2f}".format(round((x * -1000) -128, 2))

# Function to multiply ticks by 1000
def adjust_y_axis(x, pos):
    return "{:.2f}".format(round((x * 1000) - 127, 2))

In [ ]:
# File names
inpt1 = input('Please input the analysisM1 detA file: ')
inpt2 = input('Please input the analysisM1 detB file: ')
inpt3 = input('Please input the master file: ')

df_A = pd.read_csv(inpt1)
df_B = pd.read_csv(inpt2)

df_master = pd.read_csv(inpt3)
df_master = df_master.iloc[:, 1:]

# Find XRD peaks (detA - detB)
df_diff = abs(df_A - df_B)
df_param = df_master.iloc[:, [0, 1, 2, 3, 4, 5]]

# Merge everything (i.e: df_result == 'MASTER FILE')
df_result = pd.concat([df_param, df_diff], axis=1)
df_result.columns = [col if idx < 6 else 'Δ_' + col for idx, col in enumerate(df_result.columns)]

In [ ]:
### MAGNESIUM ####
# Magnesium XRD map
x = df_result['x'].values
y = df_result['y'].values
z = df_result['Δ_Mg peak area'].values

f, ax = plt.subplots(figsize=(14, 10))

Nx = 129
Ny = 129
NL = 25

# Assuming 'x', 'y', and 'magnitude' are your data arrays
xv = np.linspace(min(x), max(x), Nx)
yv = np.linspace(min(y), max(y), Ny)
xq, yq = np.meshgrid(xv, yv)

maggrid = griddata((x.flatten(), y.flatten()), z.flatten(), (xq, yq), method='linear');

contourf_plot = plt.contourf(xq, yq, maggrid, NL, cmap='nipy_spectral');

plt.text(-0.129, 0.134, str('M1'), color='red', size=50)

# Add colorbars
colorbar_contourf = plt.colorbar(contourf_plot, ax=ax, label='Contourf Colorbar')
label_text = r'$\Delta$ Mg peak'
colorbar_contourf.set_label(label_text, size=40, labelpad=10)
colorbar_contourf.ax.tick_params(labelsize=12)  # Adjust tick label size

# Axes and labels Settings
ax.xaxis.set_major_formatter(FuncFormatter(adjust_x_axis))
ax.yaxis.set_major_formatter(FuncFormatter(adjust_y_axis))
ax.tick_params(axis='both', colors='black', labelsize=16, pad=10)

plt.xlim(-0.128, -0.143);
plt.ylim(0.127, 0.135);
plt.xlabel('x coordinate [mm]', size=20, labelpad=20);
plt.ylabel('y coordinate [mm]', size=20, labelpad=20);
legend = plt.legend(prop={'size':20}, title='Dourbes - all PMCs')
legend.get_title().set_fontsize('25')
plt.grid(alpha=0.2)

# Save Mg XRD map (SVG/JPG)
plt.savefig('Mg_XRDmap(M1).svg', format='svg', bbox_inches='tight')
plt.savefig('Mg_XRDmap(M1).jpg', format='jpg', bbox_inches='tight')